In [1]:
from __future__ import print_function, division

# import sys
# sys.path.append(root_folder)
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.nn.functional as F
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import pathlib

from models.convengers import Thor, IronMan, Convengers_Cat
from models.solver import NickFury
from utils import dictionary
from utils import visualizations

plt.ion()   # interactive mode


%load_ext autoreload
%autoreload 2

In [2]:
dataset_folder = "data/tiny-imagenet-200/"
data_dir = pathlib.Path(dataset_folder)

# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(256),
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [3]:
image_datasets = {x: datasets.ImageFolder(data_dir / x, data_transforms[x]) for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=20, shuffle=True, pin_memory=True) for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
resnet_new = Thor()
ResNet_solver = NickFury(resnet_new, dataloaders, dataset_sizes)

resnet_criterion = nn.CrossEntropyLoss()
resnet_optimizer = optim.Adam(resnet_new.parameters(), lr=0.001)
resnet_exp_lr_scheduler = lr_scheduler.StepLR(resnet_optimizer, step_size=7, gamma=0.1)

In [ ]:
resnet_loss_history = ResNet_solver.train(resnet_optimizer, resnet_criterion, resnet_exp_lr_scheduler)

In [5]:
inception_new = IronMan()
inception_solver = NickFury(inception_new, dataloaders, dataset_sizes)

inception_criterion = nn.CrossEntropyLoss()
inception_optimizer = optim.Adam(inception_new.parameters(), lr=0.001)
inception_exp_lr_scheduler = lr_scheduler.StepLR(inception_optimizer, step_size=7, gamma=0.1)

In [15]:
concat_conv = Convengers_Cat(requires_grad=True)

4096


In [17]:
inputs, classes = next(iter(dataloaders['train']))
class_preds = concat_conv(inputs)

torch.Size([20, 4096, 1, 1])


In [18]:
print(classes)
print(torch.argmax(class_preds,dim=1))

tensor([ 34, 123, 190,  42,  69,  55,  98, 100,  84,  51, 118,  72, 145, 120,
        119,   5, 123,  81,   9, 142])
tensor([121,   9,  45,  49,  45,  33,   4, 163, 120, 193,  16, 159,  75, 159,
         49, 156,   0, 167,   1,  28])
